In [181]:
import pandas as pd
import re
import json
import numpy as np
from matplotlib.pyplot import figure, show
import jellyfish

# Reading the data

First we are going to read the data. 


In [182]:
authors = pd.read_csv('~/data/authors_1019.csv', encoding = "ISO-8859-1")
investigators = pd.read_csv('~/data/investigators_837.csv', encoding = "ISO-8859-1")

# First Look 
In this sections I will just take a first look at the data, check if anything is weird or not normal.

In [183]:
authors.head()

,id,first_name,middle_name,lastname,coauthor_lastnames,topics,cities,countries,orcid
0,1,kalid,NaN,abdlslam,"[""howard"",""thumbikat"",""bonnaire"",""oleksak"",""sa...","[""Female"",""Magnetic Resonance Imaging"",""Spondy...","[{""identifier"":2638077,""name"":""Sheffield""}]","[{""identifier"":2635167,""name"":""United Kingdom""}]",NaN
1,2,p,d,abel,"[""ockrim"",""lalani"",""pollock"",""price"",""langley""...","[""Risk Factors"",""Humans"",""Fractures, Spontaneo...","[{""identifier"":2643743,""name"":""London""}]","[{""identifier"":2635167,""name"":""United Kingdom""}]",NaN
2,3,k,NaN,abida,"[""syed"",""arshad"",""minakshi""]","[""Tomography, X-Ray Computed"",""Male"",""Parapare...","[{""identifier"":null,""name"":null}]","[{""identifier"":null,""name"":null}]",NaN
3,4,a,NaN,walker,"[""abou raya"",""helmii""]","[""Geriatric Assessment"",""Case-Control Studies""...","[{""identifier"":361058,""name"":""Alexandria""}]","[{""identifier"":357994,""name"":""Egypt""}]",NaN
4,5,steven,a,abrams,"[""perez"",""ellis"",""loddeke"",""shypailo"",""dawson ...","[""Time Factors"",""Adrenal Cortex Hormones"",""Chi...","[{""identifier"":4699066,""name"":""Houston""}]","[{""identifier"":6252001,""name"":""United States""}]",NaN


In [184]:
investigators.head()

,id,first_name,middle_name,lastname,coinvestigator_lastnames,topics,cities,countries,orcid
0,1,c,NaN,walker,"[""brazil"",""ballerup"",""riis"",""herriot""]","[""Calcitonin Gene-Related Peptide"",""Calcitonin...","[{""identifier"":null,""name"":null}]","[{""identifier"":null,""name"":null}]",NaN
1,2,kantonsspital,NaN,aarau,"[""gross""]","[""Osteoporotic Fractures"",""Osteoporosis, Postm...","[{""identifier"":null,""name"":null}]","[{""identifier"":null,""name"":null}]",NaN
2,3,kantonspital,NaN,wang,"[""aebi"",""di bergamo"",""bern"",""bergonie"",""vergin...","[""Tamoxifen"",""Osteoporosis"",""Breast Neoplasms""]","[{""identifier"":null,""name"":null}]","[{""identifier"":null,""name"":null}]",NaN
3,4,m,NaN,li,"[""sherman""]","[""Osteoporosis""]","[{""identifier"":null,""name"":null}]","[{""identifier"":null,""name"":null}]",NaN
4,5,paul,NaN,abel,"[""nhs""]","[""Estradiol"",""Goserelin"",""Osteoporosis"",""Estro...","[{""identifier"":2643743,""name"":""London""}]","[{""identifier"":2635167,""name"":""United Kingdom""}]",NaN


The cities and country seems to be in JSON format. Just incase we will use them later I would like to do some processing of them. Also I take notice of list format of Topic and co-authors but this can stay as is.

In [185]:
authors['cities'] = authors['cities'].apply(lambda x : json.loads(x))
authors['countries'] = authors['countries'].apply(lambda x : json.loads(x))
authors = pd.concat([authors, authors['cities'].apply(pd.Series)[0].apply(pd.Series)], axis = 1).drop('cities', axis = 1).rename(columns={"identifier": "city_id", "name": "city_name"})
authors = pd.concat([authors, authors['countries'].apply(pd.Series)[0].apply(pd.Series)], axis = 1).drop('countries', axis = 1).rename(columns={"identifier": "country_id", "name": "country_name"})

In [186]:
investigators['cities'] = investigators['cities'].apply(lambda x : json.loads(x))
investigators['countries'] = investigators['countries'].apply(lambda x : json.loads(x))
investigators = pd.concat([investigators, investigators['cities'].apply(pd.Series)[0].apply(pd.Series)], axis = 1).drop('cities', axis = 1).rename(columns={"identifier": "city_id", "name": "city_name"})
investigators = pd.concat([investigators, investigators['countries'].apply(pd.Series)[0].apply(pd.Series)], axis = 1).drop('countries', axis = 1).rename(columns={"identifier": "country_id", "name": "country_name"})

Lets take a look at the data after we have processed it just to keep it in mind.

In [187]:
authors

,id,first_name,middle_name,lastname,coauthor_lastnames,topics,orcid,city_id,city_name,country_id,country_name
0,1,kalid,NaN,abdlslam,"[""howard"",""thumbikat"",""bonnaire"",""oleksak"",""sa...","[""Female"",""Magnetic Resonance Imaging"",""Spondy...",NaN,2638077.0,Sheffield,2635167.0,United Kingdom
1,2,p,d,abel,"[""ockrim"",""lalani"",""pollock"",""price"",""langley""...","[""Risk Factors"",""Humans"",""Fractures, Spontaneo...",NaN,2643743.0,London,2635167.0,United Kingdom
2,3,k,NaN,abida,"[""syed"",""arshad"",""minakshi""]","[""Tomography, X-Ray Computed"",""Male"",""Parapare...",NaN,NaN,None,NaN,None
3,4,a,NaN,walker,"[""abou raya"",""helmii""]","[""Geriatric Assessment"",""Case-Control Studies""...",NaN,361058.0,Alexandria,357994.0,Egypt
4,5,steven,a,abrams,"[""perez"",""ellis"",""loddeke"",""shypailo"",""dawson ...","[""Time Factors"",""Adrenal Cortex Hormones"",""Chi...",NaN,4699066.0,Houston,6252001.0,United States
5,6,stefan,NaN,majumdar,"[""zaman"",""thurlimann"",""brauchli"",""pagani"",""lut...","[""Tamoxifen"",""Antineoplastic Combined Chemothe...",653524.0,2659811.0,Luzern,2658434.0,Switzerland
6,7,imelda,NaN,xu,"[""eastell"",""todd"",""schollum"",""li yu"",""wijanto""...","[""Animals"",""Middle Aged"",""Indonesia"",""Food, Fo...",NaN,NaN,None,NaN,None
7,8,antonio,j,aguiar,"[""vulcano"",""teixeira"",""rahal"",""pereira júnior""...","[""Animals, Wild"",""Maxillary Fractures"",""Animal...",NaN,NaN,None,NaN,None
8,9,i,NaN,majumdar,"[""nuñez roldan""]","[""Female"",""Parathyroid Hormone"",""Postoperative...",NaN,NaN,None,NaN,None
9,10,o,NaN,alade,"[""jackson hutson"",""rochman""]","[""Ilizarov Technique"",""Male"",""Debridement"",""Fe...",NaN,NaN,None,NaN,None


In [188]:
investigators

,id,first_name,middle_name,lastname,coinvestigator_lastnames,topics,orcid,city_id,city_name,country_id,country_name
0,1,c,NaN,walker,"[""brazil"",""ballerup"",""riis"",""herriot""]","[""Calcitonin Gene-Related Peptide"",""Calcitonin...",NaN,NaN,None,NaN,None
1,2,kantonsspital,NaN,aarau,"[""gross""]","[""Osteoporotic Fractures"",""Osteoporosis, Postm...",NaN,NaN,None,NaN,None
2,3,kantonspital,NaN,wang,"[""aebi"",""di bergamo"",""bern"",""bergonie"",""vergin...","[""Tamoxifen"",""Osteoporosis"",""Breast Neoplasms""]",NaN,NaN,None,NaN,None
3,4,m,NaN,li,"[""sherman""]","[""Osteoporosis""]",NaN,NaN,None,NaN,None
4,5,paul,NaN,abel,"[""nhs""]","[""Estradiol"",""Goserelin"",""Osteoporosis"",""Estro...",NaN,2643743.0,London,2635167.0,United Kingdom
5,6,anna,NaN,walker,[],"[""Vitamins"",""Osteoporosis"",""Ergocalciferols"",""...",NaN,361058.0,Alexandria,357994.0,Egypt
6,7,s,NaN,abrams,"[""abrams""]","[""Calcium, Dietary"",""Osteoporosis"",""Puberty, P...",NaN,4699066.0,Houston,6252001.0,United States
7,8,steve,NaN,abrams,"[""abrams""]","[""Calcium, Dietary"",""Osteoporosis"",""Puberty, P...",NaN,NaN,None,NaN,None
8,9,o,NaN,adeyemi,"[""rothstein""]","[""Osteoporosis"",""Bone Diseases"",""HIV Infection...",NaN,NaN,None,NaN,None
9,10,fibrosi,NaN,adulti,"[""cistica"",""bianchi""]","[""Cystic Fibrosis"",""Fibrosis"",""Osteoporosis"",""...",NaN,NaN,None,NaN,None


# Divide and conquer
I have philosophy when it comes to problems in data science. KISS (Keep It Simple Stupid). With a cartisean product of the subset of the data that we have ground truth for, it would be possible to engineer some features from the strings and build a predictive model. However I think there are better ways to get the ball rolling. We will try a divide and conquer techique. We will try to develop unique IDs using information in the data.

1. Join data that has ORCIDs
2. Remove data with ORCIDs from the data
3. Find unique last names and join those from the two datasets
4. Concatenate orcid join and unique last name join
5. Filter out already joined data from the data
6. Find unique combination of the first letter from first name and last name
7. Join on this combination and concatenate with previous data.
8. Filter out already joined data from the data
9. Combine fist, middle, and last name into 1 column for remaining data
10. Do fuzzy join
11. Combine with previously joined data

In [189]:
investigators_with_orcid = investigators[investigators['orcid'].notnull()].rename(columns={'id':'id_authors'})[['id_authors', 'orcid']]
authors_with_orcid = authors[authors['orcid'].notnull()].rename(columns={'id':'id_investigators'})[['id_investigators', 'orcid']]

In [190]:
joined_on_orcid = pd.merge(investigators_with_orcid, authors_with_orcid, on='orcid')[['id_authors', 'id_investigators']]

In [191]:
# finding authors that do not have orcid
authors_without_orcid = authors[authors['orcid'].isnull()]
investigators_without_orcid = investigators[investigators['orcid'].isnull()]
    
# finding authors with unique last names
authors_unique_last_name = authors_without_orcid.drop_duplicates(subset=['lastname'])
investigators_unique_last_name = investigators_without_orcid.drop_duplicates(subset=['lastname'])

In [192]:
#unique last names count for authors
(authors_unique_last_name.lastname.value_counts().apply(pd.Series)[0] == 1).value_counts()


True    519
Name: 0, dtype: int64

In [193]:
#unique last names count for investigators
(investigators_unique_last_name.lastname.value_counts().apply(pd.Series)[0] == 1).value_counts()


True    428
Name: 0, dtype: int64

In [194]:
joined_on_unique_names = pd.merge(authors_unique_last_name, investigators_unique_last_name, on='lastname',
                                  suffixes=('_authors', '_investigators'))[['id_authors', 'id_investigators']]

In [195]:
#198 more joined! It means however there is not perfect overlap with the data
joined_on_unique_names.nunique()

id_authors          198
id_investigators    198
dtype: int64

In [196]:
concat_orcid_unique_names = pd.concat([joined_on_unique_names, joined_on_orcid])

In [197]:
# Removing authors that have already been sucessfully joined
authors_left = authors[~authors['id'].isin(concat_orcid_unique_names['id_authors'])]
investigators_left = investigators[~investigators['id'].isin(concat_orcid_unique_names['id_investigators'])]

In [199]:
# Creating first letter last name combo
authors_left['first_letter_last_name'] = (authors_left.first_name.str[0] + '_' + authors_left.lastname).astype(str)
investigators_left['first_letter_last_name'] = (investigators_left.first_name.str[0] + '_' + investigators_left.lastname).astype(str)
authors_left_unique_combo = authors_left.drop_duplicates(subset=['first_letter_last_name'])
investigators_left_unique_combo = investigators_left.drop_duplicates(subset=['first_letter_last_name'])

In [200]:
joined_on_unique_names_combo = pd.merge(authors_left_unique_combo, investigators_left_unique_combo, on='first_letter_last_name',
                                  suffixes=('_authors', '_investigators'))[['id_authors', 'id_investigators']]

In [201]:
joined_on_unique_names_combo.shape

(180, 2)

In [202]:
concat_all_before_fuzzy = pd.concat([concat_orcid_unique_names,joined_on_unique_names_combo])

In [203]:
# Removing authors that have already been sucessfully joined
authors_for_fuzzy_join = authors[~authors['id'].isin(concat_all_before_fuzzy['id_authors'])]
investigators_for_fuzzy_join = investigators[~investigators['id'].isin(concat_all_before_fuzzy['id_investigators'])]

In [204]:
authors_for_fuzzy_join.shape

(485, 11)

In [205]:
# Replace middle name with neatural character
authors_for_fuzzy_join['middle_name'] = authors_for_fuzzy_join.middle_name.fillna('_')
investigators_for_fuzzy_join['middle_name'] = investigators_for_fuzzy_join.middle_name.fillna('_')

In [206]:
# Creating fuzzy join ID
authors_for_fuzzy_join['fuzzy_id'] = (authors_for_fuzzy_join.first_name + '_' + authors_for_fuzzy_join.middle_name + "_" + authors_for_fuzzy_join.lastname).astype(str)
investigators_for_fuzzy_join['fuzzy_id'] = (investigators_for_fuzzy_join.first_name + '_' + investigators_for_fuzzy_join.middle_name + "_" + investigators_for_fuzzy_join.lastname).astype(str)

In [207]:
# Renaming and selecting columns
authors_for_fuzzy_join = authors_for_fuzzy_join[['fuzzy_id', 'id']].rename(columns={'id':'id_authors'})
investigators_for_fuzzy_join = investigators_for_fuzzy_join[['fuzzy_id', 'id']].rename(columns={'id':'id_investigators'})

In [208]:
# get closest match algorithm based on jaro_winkler
def get_closest_match(x, list_strings):
    best_match = None
    highest_jw = 0
    for current_string in list_strings:
        current_score = jellyfish.jaro_winkler(x, current_string)
        
        if(current_score > highest_jw):
            highest_jw = current_score
            best_match = current_string
            
    return best_match

# Evaluation

In [209]:
investigators_with_orcid = investigators[investigators['orcid'].notnull()].rename(columns={'id':'id_authors'})
authors_with_orcid = authors[authors['orcid'].notnull()].rename(columns={'id':'id_investigators'})

In [210]:
authors_with_orcid['fuzzy_id'] = (authors_with_orcid.first_name + '_' + authors_with_orcid.middle_name + "_" + authors_with_orcid.lastname).astype(str)
investigators_with_orcid['fuzzy_id'] = (investigators_with_orcid.first_name + '_' + investigators_with_orcid.middle_name + "_" + investigators_with_orcid.lastname).astype(str)

In [211]:
authors_with_orcid.fuzzy_id = authors_with_orcid.fuzzy_id.map(lambda x: get_closest_match(x, investigators_with_orcid.fuzzy_id))

In [212]:
evaluate_fuzz = pd.merge(authors_with_orcid, investigators_with_orcid, how='left', on='fuzzy_id', suffixes=('_authors', '_investigators'))

In [213]:
(evaluate_fuzz['orcid_authors'] == evaluate_fuzz['orcid_investigators']).sum()

133

In [214]:
authors_with_orcid.shape

(206, 12)

In [215]:
#correct matches using fuzzy join
133/206

0.6456310679611651

In [216]:
investigators_for_fuzzy_join.fuzzy_id = investigators_for_fuzzy_join.fuzzy_id.map(lambda x: get_closest_match(x, authors_for_fuzzy_join.fuzzy_id))


In [217]:
fuzzy = pd.merge(authors_for_fuzzy_join, investigators_for_fuzzy_join, on='fuzzy_id')


In [218]:
final_fuzz = fuzzy[['id_authors', 'id_investigators']]

In [219]:
concat_all = pd.concat([final_fuzz, concat_all_before_fuzzy])

In [220]:
concat_all.shape

(920, 2)

# Final Remarks
Given the time constains I could not try everything that I would have liked. I would still investigate further the approached outside of ML further.I would test different distance calculations, as well as different variable combinations to make a fuzzy id out of.

From the machine learning perspective I would have done a cartisean product of ground truth data, feature engineer label, plus feature engineer multiple features related to distance of strings. I would test out different string distance metrics. I would start with a base_line logistics regression, then move to more complex models from there. If I engineer to many distance features I may try to do a PCA.

From a code perspecive, way to much duplicated code, everything needs to be put into functions. As well as proper unit testing for little pieces of the code.